In [1]:
import pandas as pd
import requests
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#url de la API
api_url='https://api.yelp.com/v3/businesses/search'

#estos datos corresponden a una cuenta de usuario creada previamente
clientid='GWOCZh9-BmZxtdsAjr7Gug'
apikey='FHVvXoNmTXIl9DuxYis7AV5uLPujm9MLwrhgs5NgvCfaOxd3V6mxt6dQU8eEqYJiGxe816XATx7ufWjbMWqbV-2Uku1jxBJv8BGRC74NroLPl27PDQqs0tDixit-YHYx'
headers={'Authorization':'Bearer %s'%apikey}

In [4]:
ciudad = "miami" # Se puede elegir cualquier ciudad del dataset entregado

In [5]:
# De esta manera se consulta el API de Yelp para obtener información sobre restaurantes en la ciudad especificada
params={'term':'restaurants','location': ciudad,'limit':50}
response=requests.get(api_url,params=params,headers=headers)
data=response.json()
data

{'businesses': [{'id': 'K3ukx2e11xTRtYBU01dmrA',
   'alias': 'salty-flame-miami',
   'name': 'Salty Flame',
   'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/_6wShTvzfucB5dxFZv4Mpg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/salty-flame-miami?adjust_creative=GWOCZh9-BmZxtdsAjr7Gug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GWOCZh9-BmZxtdsAjr7Gug',
   'review_count': 198,
   'categories': [{'alias': 'asianfusion', 'title': 'Asian Fusion'},
    {'alias': 'steak', 'title': 'Steakhouses'},
    {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
   'rating': 4.4,
   'coordinates': {'latitude': 25.76022, 'longitude': -80.19267},
   'transactions': ['delivery', 'pickup'],
   'location': {'address1': '1414 Brickell Ave',
    'address2': None,
    'address3': '',
    'city': 'Miami',
    'zip_code': '33131',
    'country': 'US',
    'state': 'FL',
    'display_address': ['1414 Brickell Ave', 'Miami, FL 33131']},
   'phone': '+1305563897

In [6]:
#Veamos los keys del diccionario recibido
data.keys()

dict_keys(['businesses', 'total', 'region'])

In [7]:
#El primer elemento del diccionario indica el total de restaurants existentes en la API
print('En total la base de datos registra %d restaurants'%data['total'])

En total la base de datos registra 5600 restaurants


In [8]:
data['businesses'] # Hasta acá se le entregaría a los estudiantes

[{'id': 'K3ukx2e11xTRtYBU01dmrA',
  'alias': 'salty-flame-miami',
  'name': 'Salty Flame',
  'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/_6wShTvzfucB5dxFZv4Mpg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/salty-flame-miami?adjust_creative=GWOCZh9-BmZxtdsAjr7Gug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GWOCZh9-BmZxtdsAjr7Gug',
  'review_count': 198,
  'categories': [{'alias': 'asianfusion', 'title': 'Asian Fusion'},
   {'alias': 'steak', 'title': 'Steakhouses'},
   {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
  'rating': 4.4,
  'coordinates': {'latitude': 25.76022, 'longitude': -80.19267},
  'transactions': ['delivery', 'pickup'],
  'location': {'address1': '1414 Brickell Ave',
   'address2': None,
   'address3': '',
   'city': 'Miami',
   'zip_code': '33131',
   'country': 'US',
   'state': 'FL',
   'display_address': ['1414 Brickell Ave', 'Miami, FL 33131']},
  'phone': '+13055638972',
  'display_phone': '(305) 563-89

Traemos los restaurantes actualizados

In [9]:
lista_restaurantes = []

# Bucle con offset para paginación
offset = 0
limite = 50
max_registros = 1000 # puedes subirlo si quieres traer más (máximo 1000 en Yelp API)

while offset < max_registros:
    params = { 'term': 'restaurants',
              'location': ciudad,
              'limit': limite,
              'offset': offset}

    response = requests.get(api_url, params=params, headers=headers)
    data = response.json()

    # Verificamos si hay resultados
    data_businesses = data.get('businesses', [])
    if not data_businesses:
        print(f"No mas resultados {offset}. deten el bucle.")
        break

    # Normalización de los datos
    df_restaurantes = pd.json_normalize(
        data_businesses,
        sep='_',
        record_path=['categories'],  # Desempaqueta categorías
        meta=[
            'id',
            'alias',
            'name',
            'price',
            'rating',
            'review_count',
            'distance',
            ['coordinates', 'latitude'],
            ['coordinates', 'longitude'],
            ['location', 'address1'],
            ['location', 'city']
        ],
        meta_prefix='business_',  # Prefijo para evitar conflictos de nombres
        errors='ignore'  # Ignora claves faltantes y las rellena con NaN
    )

    lista_restaurantes.append(df_restaurantes)

    print(f"{len(data_businesses)} results added at offset {offset}.")
    offset += limite  # pasamos a la siguiente página

# Concatenar todos los resultados en un único DataFrame
df_final = pd.concat(lista_restaurantes, ignore_index=True)

# Mostramos el tamaño final
print("DataFrame created with", len(df_final), "rows.")

# Ver las primeras filas
print(df_final.head())

50 results added at offset 0.
50 results added at offset 50.
50 results added at offset 100.
50 results added at offset 150.
No mas resultados 200. deten el bucle.
DataFrame created with 447 rows.
          alias          title             business_id  \
0   asianfusion   Asian Fusion  K3ukx2e11xTRtYBU01dmrA   
1         steak    Steakhouses  K3ukx2e11xTRtYBU01dmrA   
2  cocktailbars  Cocktail Bars  K3ukx2e11xTRtYBU01dmrA   
3         cuban          Cuban  UXHxLN3DcDGI57uDIfCuJA   
4          bars           Bars  UXHxLN3DcDGI57uDIfCuJA   

                           business_alias                    business_name  \
0                       salty-flame-miami                      Salty Flame   
1                       salty-flame-miami                      Salty Flame   
2                       salty-flame-miami                      Salty Flame   
3  olds-havana-cuban-bar-and-cocina-miami  Old's Havana Cuban Bar & Cocina   
4  olds-havana-cuban-bar-and-cocina-miami  Old's Havana Cuban Ba

In [10]:
df_restaurantes_miami = df_final.copy()
df_restaurantes_miami

,alias,title,business_id,business_alias,business_name,business_price,business_rating,business_review_count,business_distance,business_coordinates_latitude,business_coordinates_longitude,business_location_address1,business_location_city
0,asianfusion,Asian Fusion,K3ukx2e11xTRtYBU01dmrA,salty-flame-miami,Salty Flame,NaN,4.4,198,9568.050891,25.76022,-80.19267,1414 Brickell Ave,Miami
1,steak,Steakhouses,K3ukx2e11xTRtYBU01dmrA,salty-flame-miami,Salty Flame,NaN,4.4,198,9568.050891,25.76022,-80.19267,1414 Brickell Ave,Miami
2,cocktailbars,Cocktail Bars,K3ukx2e11xTRtYBU01dmrA,salty-flame-miami,Salty Flame,NaN,4.4,198,9568.050891,25.76022,-80.19267,1414 Brickell Ave,Miami
3,cuban,Cuban,UXHxLN3DcDGI57uDIfCuJA,olds-havana-cuban-bar-and-cocina-miami,Old's Havana Cuban Bar & Cocina,$$,4.4,3012,7035.90542,25.765594,-80.219238,1442 SW 8th St,Miami
4,bars,Bars,UXHxLN3DcDGI57uDIfCuJA,olds-havana-cuban-bar-and-cocina-miami,Old's Havana Cuban Bar & Cocina,$$,4.4,3012,7035.90542,25.765594,-80.219238,1442 SW 8th St,Miami
5,venues,Venues & Event Spaces,UXHxLN3DcDGI57uDIfCuJA,olds-havana-cuban-bar-and-cocina-miami,Old's Havana Cuban Bar & Cocina,$$,4.4,3012,7035.90542,25.765594,-80.219238,1442 SW 8th St,Miami
6,italian,Italian,oxtMfBGmVNE18pFVuw7lFg,fratellino-coral-gables-2,Fratellino,$$$,4.8,1815,2834.064937,25.74917,-80.26011,264 Miracle Mile,Coral Gables
7,mediterranean,Mediterranean,ix8ifP1jQM9ektdVAs19sQ,crazy-about-you-miami-3,Crazy About You,$$,4.2,2797,9973.468955,25.762015,-80.188986,1155 Brickell Bay Dr,Miami
8,italian,Italian,ix8ifP1jQM9ektdVAs19sQ,crazy-about-you-miami-3,Crazy About You,$$,4.2,2797,9973.468955,25.762015,-80.188986,1155 Brickell Bay Dr,Miami
9,spanish,Spanish,ix8ifP1jQM9ektdVAs19sQ,crazy-about-you-miami-3,Crazy About You,$$,4.2,2797,9973.468955,25.762015,-80.188986,1155 Brickell Bay Dr,Miami


In [11]:
df_restaurantes_miami.describe()

,alias,title,business_id,business_alias,business_name,business_price,business_rating,business_review_count,business_distance,business_coordinates_latitude,business_coordinates_longitude,business_location_address1,business_location_city
count,447,447,447,447,447,307,447.0,447,447.000000,447.00000,447.00000,447,447
unique,103,103,200,200,199,4,18.0,175,200.000000,200.00000,200.00000,196,10
top,seafood,Seafood,K3ukx2e11xTRtYBU01dmrA,salty-flame-miami,CVI.CHE 105,$$,4.4,1,9568.050891,25.76022,-80.19267,3555 SW 8th St,Miami
freq,25,25,3,3,5,222,80.0,18,3.000000,3.00000,3.00000,5,318


In [12]:
df_restaurantes_miami.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   alias                           447 non-null    object
 1   title                           447 non-null    object
 2   business_id                     447 non-null    object
 3   business_alias                  447 non-null    object
 4   business_name                   447 non-null    object
 5   business_price                  307 non-null    object
 6   business_rating                 447 non-null    object
 7   business_review_count           447 non-null    object
 8   business_distance               447 non-null    object
 9   business_coordinates_latitude   447 non-null    object
 10  business_coordinates_longitude  447 non-null    object
 11  business_location_address1      447 non-null    object
 12  business_location_city          447 non-null    ob

In [13]:
df_restaurantes_miami = df_restaurantes_miami.astype({
    'business_rating': 'float',
    'business_review_count': 'int',
    'business_distance' : 'float',
    'business_coordinates_latitude' : 'float',
    'business_coordinates_longitude' : 'float'
})

In [14]:
df_restaurantes_miami.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   alias                           447 non-null    object 
 1   title                           447 non-null    object 
 2   business_id                     447 non-null    object 
 3   business_alias                  447 non-null    object 
 4   business_name                   447 non-null    object 
 5   business_price                  307 non-null    object 
 6   business_rating                 447 non-null    float64
 7   business_review_count           447 non-null    int64  
 8   business_distance               447 non-null    float64
 9   business_coordinates_latitude   447 non-null    float64
 10  business_coordinates_longitude  447 non-null    float64
 11  business_location_address1      447 non-null    object 
 12  business_location_city          447 

In [15]:
# Agrupar por ID y concatenar los titles en una columna renombrada como types con el fin de no perder la información adicional para utilidad en futuras instancias
df_tipos = df_restaurantes_miami.groupby("business_id")["title"].agg(lambda x: ", ".join(x)).reset_index()
df_tipos.rename(columns = {'title':'types'},inplace=True)


In [16]:
# unión de los data frames para obtener en el original la columna creada de forma previa
df_restaurantes_miami= df_restaurantes_miami.merge(df_tipos, on = 'business_id')
df_restaurantes_miami

,alias,title,business_id,business_alias,business_name,business_price,business_rating,business_review_count,business_distance,business_coordinates_latitude,business_coordinates_longitude,business_location_address1,business_location_city,types
0,asianfusion,Asian Fusion,K3ukx2e11xTRtYBU01dmrA,salty-flame-miami,Salty Flame,NaN,4.4,198,9568.050891,25.760220,-80.192670,1414 Brickell Ave,Miami,"Asian Fusion, Steakhouses, Cocktail Bars"
1,steak,Steakhouses,K3ukx2e11xTRtYBU01dmrA,salty-flame-miami,Salty Flame,NaN,4.4,198,9568.050891,25.760220,-80.192670,1414 Brickell Ave,Miami,"Asian Fusion, Steakhouses, Cocktail Bars"
2,cocktailbars,Cocktail Bars,K3ukx2e11xTRtYBU01dmrA,salty-flame-miami,Salty Flame,NaN,4.4,198,9568.050891,25.760220,-80.192670,1414 Brickell Ave,Miami,"Asian Fusion, Steakhouses, Cocktail Bars"
3,cuban,Cuban,UXHxLN3DcDGI57uDIfCuJA,olds-havana-cuban-bar-and-cocina-miami,Old's Havana Cuban Bar & Cocina,$$,4.4,3012,7035.905420,25.765594,-80.219238,1442 SW 8th St,Miami,"Cuban, Bars, Venues & Event Spaces"
4,bars,Bars,UXHxLN3DcDGI57uDIfCuJA,olds-havana-cuban-bar-and-cocina-miami,Old's Havana Cuban Bar & Cocina,$$,4.4,3012,7035.905420,25.765594,-80.219238,1442 SW 8th St,Miami,"Cuban, Bars, Venues & Event Spaces"
5,venues,Venues & Event Spaces,UXHxLN3DcDGI57uDIfCuJA,olds-havana-cuban-bar-and-cocina-miami,Old's Havana Cuban Bar & Cocina,$$,4.4,3012,7035.905420,25.765594,-80.219238,1442 SW 8th St,Miami,"Cuban, Bars, Venues & Event Spaces"
6,italian,Italian,oxtMfBGmVNE18pFVuw7lFg,fratellino-coral-gables-2,Fratellino,$$$,4.8,1815,2834.064937,25.749170,-80.260110,264 Miracle Mile,Coral Gables,Italian
7,mediterranean,Mediterranean,ix8ifP1jQM9ektdVAs19sQ,crazy-about-you-miami-3,Crazy About You,$$,4.2,2797,9973.468955,25.762015,-80.188986,1155 Brickell Bay Dr,Miami,"Mediterranean, Italian, Spanish"
8,italian,Italian,ix8ifP1jQM9ektdVAs19sQ,crazy-about-you-miami-3,Crazy About You,$$,4.2,2797,9973.468955,25.762015,-80.188986,1155 Brickell Bay Dr,Miami,"Mediterranean, Italian, Spanish"
9,spanish,Spanish,ix8ifP1jQM9ektdVAs19sQ,crazy-about-you-miami-3,Crazy About You,$$,4.2,2797,9973.468955,25.762015,-80.188986,1155 Brickell Bay Dr,Miami,"Mediterranean, Italian, Spanish"


In [17]:
#eliminando ID duplicados para tener datos más limpios
df_restaurantes_miami = df_restaurantes_miami.drop_duplicates('business_id')

In [18]:
#obteniendo promedios que me resultan relevantes para evaluar el performance de los restaurantes
promedio_business_rating = (df_restaurantes_miami.groupby(['title', 'business_price'])['business_rating'].mean())
promedio_business_reviews = (df_restaurantes_miami.groupby(['title', 'business_price'])['business_review_count'].mean())

In [19]:
# función que evalua si los restaurantes igualan o superan los promedios relevantes y los categoriza como Top Restaurant de forma verdaddera o falsa en la columna Top Restaurant per title
def es_top_restaurant(row):
    key = (row['title'], row['business_price'])
    return (
        (row['business_rating'] >= promedio_business_rating.get(key, 0)) and
        (row['business_review_count'] >= promedio_business_reviews.get(key, 0))
    )

df_restaurantes_miami['top_restaurant_per_title'] = df_restaurantes_miami.apply(es_top_restaurant, axis=1)

C:\Users\alex_\AppData\Local\Temp\ipykernel_37812\3953376510.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_restaurantes_miami['top_restaurant_per_title'] = df_restaurantes_miami.apply(es_top_restaurant, axis=1)


In [20]:
df_restaurantes_miami.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 0 to 445
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   alias                           200 non-null    object 
 1   title                           200 non-null    object 
 2   business_id                     200 non-null    object 
 3   business_alias                  200 non-null    object 
 4   business_name                   200 non-null    object 
 5   business_price                  139 non-null    object 
 6   business_rating                 200 non-null    float64
 7   business_review_count           200 non-null    int64  
 8   business_distance               200 non-null    float64
 9   business_coordinates_latitude   200 non-null    float64
 10  business_coordinates_longitude  200 non-null    float64
 11  business_location_address1      200 non-null    object 
 12  business_location_city          200 non-n

A pesar de que se indican que 66 restaurantes no tienen el dato de business price no se logra identificar una manera clara de poder completar el dato de forma que no ocasione un sesgo marcado en la información, por tanto se opta con dejarlos como 'sin dato' y buscar alguna forma de trabajar con esa información más adelante

In [21]:
df_restaurantes_miami['business_price'] = df_restaurantes_miami['business_price'].fillna('sin dato')

C:\Users\alex_\AppData\Local\Temp\ipykernel_37812\359055235.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_restaurantes_miami['business_price'] = df_restaurantes_miami['business_price'].fillna('sin dato')


In [22]:
df_restaurantes_miami.isna().count().sort_values(ascending=True)

alias                             200
title                             200
business_id                       200
business_alias                    200
business_name                     200
business_price                    200
business_rating                   200
business_review_count             200
business_distance                 200
business_coordinates_latitude     200
business_coordinates_longitude    200
business_location_address1        200
business_location_city            200
types                             200
top_restaurant_per_title          200
dtype: int64

In [23]:
df_restaurantes_miami.to_csv('restaurantes_miami_datos_limpios')